#Install libraries

In [1]:
!pip install transformers
!pip install torch torchvision
!pip install pandas
!pip install numpy
!pip install datasets
!pip install pytorch_transformers
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

Import the required libraries

In [2]:
import torch
from torch.utils.data import (TensorDataset, DataLoader,
                              RandomSampler, SequentialSampler)

from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import BertForSequenceClassification
from pytorch_transformers import AdamW, WarmupLinearSchedule

from distutils.version import LooseVersion as LV

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
import torch.nn.functional as F
import io
from scipy.spatial.distance import mahalanobis
import pandas as pd
import numpy as np

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset

import tensorflow_datasets as tfds

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


sns.set()

if torch.cuda.is_available():
    device = torch.device('cuda')
    devicename = '['+torch.cuda.get_device_name(0)+']'
else:
    device = torch.device('cpu')
    devicename = ""
    
print('Using PyTorch version:', torch.__version__,
      'Device:', device, devicename)
assert(LV(torch.__version__) >= LV("1.0.0"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using PyTorch version: 1.13.1+cu116 Device: cuda [NVIDIA A100-SXM4-40GB]


Download the IMDb and SST-2 datasets and extract them.

In [3]:
# Load the IMDB dataset
imdb_dataset = load_dataset("imdb")


# Load the SST-2 dataset
sst2_dataset = load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Load the IMDb dataset using pandas, and preprocess the text data by removing HTML tags, non-alphanumeric characters, and stop words.

In [4]:
print(imdb_dataset.column_names)

{'train': ['text', 'label'], 'test': ['text', 'label'], 'unsupervised': ['text', 'label']}


In [5]:
# Load the IMDb dataset
imdb_df = pd.concat([pd.DataFrame(imdb_dataset['train']),pd.DataFrame(imdb_dataset['test'])])
imdb_df = imdb_df.reset_index(drop=True)


print('\nIMDB data loaded:')
print('data set:', imdb_df.shape)
print(imdb_df['label'].unique())


IMDB data loaded:
data set: (50000, 2)
[0 1]


Load the SST-2 dataset using pandas, and preprocess the text data in the same way as the IMDb dataset

In [6]:
print(sst2_dataset.column_names)

{'train': ['sentence', 'label', 'idx'], 'validation': ['sentence', 'label', 'idx'], 'test': ['sentence', 'label', 'idx']}


In [7]:
# Load the SST-2 dataset

sst2_df = pd.concat([pd.DataFrame(sst2_dataset['train'])[['sentence', 'label']],pd.DataFrame(sst2_dataset['validation'])[['sentence', 'label']]])
sst2_df = sst2_df.rename(columns={'sentence': 'text'})
sst2_df = sst2_df.reset_index(drop=True)


print('\nSST2 data loaded:')
print('data set:', sst2_df.shape)
print(sst2_df['label'].unique())



SST2 data loaded:
data set: (68221, 2)
[0 1]


In [8]:
# Preprocess the text data
sst2_df ['text'] = sst2_df ['text'].str.replace('<.*?>', '', regex=True) # remove HTML tags
sst2_df ['text'] = sst2_df ['text'].str.replace('[^a-zA-Z0-9\s]', '', regex=True) # remove non-alphanumeric characters
stop_words = set(stopwords.words('english'))
sst2_df ['text'] = sst2_df ['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words])) # remove stop words

In [9]:
# Let's view some random reviews:
print(sst2_df.sample(5))

                                                    text  label
17606                                          recommend      1
6310   accompanying stunthungry dimwits random series...      0
28241                                          year best      1
66363           one bestsustained ideas ever seen screen      1
57838  skillful filmmaker impart message without blud...      1


Split into train and test set

In [10]:
# Define your features and target variable
X = sst2_df.drop("label", axis=1)
y = sst2_df["label"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the train and test sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

sst2_train_df = pd.concat([X_train,y_train], axis=1)
sst2_test_df = pd.concat([X_test,y_test], axis=1)

print('\nSST2 data re splitted:')
print('train:', sst2_train_df.shape)
print('test:', sst2_test_df.shape)
print(sst2_train_df['label'].unique())
print(sst2_test_df['label'].unique())


X_train shape: (54576, 1)
X_test shape: (13645, 1)
y_train shape: (54576,)
y_test shape: (13645,)

SST2 data re splitted:
train: (54576, 2)
test: (13645, 2)
[1 0]
[1 0]


In [11]:
# Let's view some random reviews:
print(sst2_train_df.sample(5))
print(sst2_test_df.sample(5))

                                                    text  label
4951       laborious pacing endless exposition tightened      0
64341  suffers timid parsing barnside target sons try...      0
22474  puts suspenseful spin standard horror flick fo...      1
65203                 potential touched angel simplicity      1
8017                                      annoying score      0
                                                    text  label
26294                             ca nt help feel stoked      1
64247                heartwarming nonjudgmental kind way      1
48473                          giant pile elephant feces      0
8241                               marvelous performance      1
26610  trapped weird relative trots video took family...      0


IN-DS: SST2
OOD-DS: IMDB

In [12]:
train_df = sst2_train_df
test_df = sst2_test_df
n_ood = 0.2
ood_df = imdb_df.sample(int(n_ood*test_df.shape[0]))
ood_df = ood_df.reset_index(drop=True)

In [13]:
# Preprocess the text data
ood_df ['text'] = ood_df ['text'].str.replace('<.*?>', '', regex=True) # remove HTML tags
ood_df ['text'] = ood_df ['text'].str.replace('[^a-zA-Z0-9\s]', '', regex=True) # remove non-alphanumeric characters
stop_words = set(stopwords.words('english'))
ood_df ['text'] = ood_df ['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words])) # remove stop words


# Let's view some random reviews:
print(ood_df.sample(5))

                                                   text  label
235   I doubt real story development Western Union w...      1
2209  The thirties horror films best remembered alwa...      1
2052  Where I begin Walt Disneys happiest motion pic...      1
28    A number brides mysteriously murdered altar la...      0
437   I couldnt agree comment like Falling Peter Wel...      1


In [14]:
del X_train, X_test, y_train, y_test

The token `[CLS]` is a special token required by BERT at the beginning of the sentence.

In [15]:
sentences_train = train_df.text.values
sentences_train = ["[CLS] " + s for s in sentences_train]

sentences_test = test_df.text.values
sentences_test = ["[CLS] " + s for s in sentences_test]

sentences_ood = ood_df.text.values
sentences_ood = ["[CLS] " + s for s in sentences_ood]


labels_train = train_df.label.values
labels_test  = test_df.label.values
labels_ood  = ood_df.label.values

print ("\nThe first training sentence:")
print(sentences_train[0], 'LABEL:', labels_train[0])



The first training sentence:
[CLS] wildly alive LABEL: 1


Next we use the BERT tokenizer to convert the sentences into tokens
that match the data BERT was trained on.


In [16]:
BERTMODEL = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(BERTMODEL,
                                          do_lower_case=True)

tokenized_train = [tokenizer.tokenize(s) for s in sentences_train]
tokenized_test  = [tokenizer.tokenize(s) for s in sentences_test]
tokenized_ood  = [tokenizer.tokenize(s) for s in sentences_ood]

print ("\nThe full tokenized first training sentence:")
print (tokenized_train[0])

print ("\nThe full tokenized first test sentence:")
print (tokenized_test[0])

print ("\nThe full tokenized first OOD sentence:")
print (tokenized_ood[0])

100%|██████████| 231508/231508 [00:00<00:00, 3797150.54B/s]



The full tokenized first training sentence:
['[CLS]', 'wildly', 'alive']

The full tokenized first test sentence:
['[CLS]', 'best', 'script']

The full tokenized first OOD sentence:
['[CLS]', 'the', 'poverty', 'row', 'horror', 'pictures', '1930s', '40', '##s', 'de', '##press', 'hell', 'god', 'knows', 'i', 'nothing', 'low', '##bu', '##dget', 'films', 'ones', 'produced', 'period', 'dr', '##ear', '##y', 'sha', '##bby', 'look', 'them', '##and', 'final', 'analysis', 'aren', '##t', 'good', 'the', 'corpse', 'vanish', '##es', 'slightly', 'entertaining', 'bottom', '##oft', '##he', '##bar', '##rel', 'dr', '##eck', 'like', 'the', 'invisible', 'ghost', 'the', 'ape', 'man', 'classic', 'bela', 'lu', '##gos', '##i', 'long', 'past', 'dracula', 'hey', '##day', 'plays', 'yet', 'another', 'mad', 'doctor', 'unbearable', 'elizabeth', 'russell', 'plays', 'wife', 'they', 'sleep', 'coffin', '##s', 'lu', '##gos', '##i', 'explains', 'doubtful', 'young', 'female', 'reporter', 'lu', '##ana', 'walters', 'coffin',


Now we set the maximum sequence lengths for our training and test
sentences as `MAX_LEN_TRAIN` and `MAX_LEN_TEST`. The maximum length
supported by the used BERT model is 512.

The token `[SEP]` is another special token required by BERT at the
end of the sentence.

In [17]:
MAX_LEN_TRAIN, MAX_LEN_TEST = 128, 512

tokenized_train = [t[:(MAX_LEN_TRAIN-1)]+['SEP'] for t in tokenized_train]
tokenized_test  = [t[:(MAX_LEN_TEST-1)]+['SEP'] for t in tokenized_test]
tokenized_ood  = [t[:(MAX_LEN_TEST-1)]+['SEP'] for t in tokenized_ood]

print ("\nThe truncated tokenized first training sentence:")
print (tokenized_train[0])


The truncated tokenized first training sentence:
['[CLS]', 'wildly', 'alive', 'SEP']



Next we use the BERT tokenizer to convert each token into an integer
index in the BERT vocabulary. We also pad any shorter sequences to
`MAX_LEN_TRAIN` or `MAX_LEN_TEST` indices with trailing zeros.

In [18]:
ids_train = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_train]
ids_train = np.array([np.pad(i, (0, MAX_LEN_TRAIN-len(i)),
                             mode='constant') for i in ids_train])

ids_test = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_test]
ids_test = np.array([np.pad(i, (0, MAX_LEN_TEST-len(i)),
                            mode='constant') for i in ids_test])


ids_ood = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_ood]
ids_ood = np.array([np.pad(i, (0, MAX_LEN_TEST-len(i)),
                            mode='constant') for i in ids_ood])

print ("\nThe indices of the first training sentence:")
print (ids_train[0])


The indices of the first training sentence:
[  101 13544  4142   100     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


BERT also requires *attention masks*, with 1 for each real token in
the sequences and 0 for the padding:

In [19]:
amasks_train, amasks_test , amasks_ood = [], [] , []

for seq in ids_train:
  seq_mask = [float(i>0) for i in seq]
  amasks_train.append(seq_mask)

for seq in ids_test:
  seq_mask = [float(i>0) for i in seq]
  amasks_test.append(seq_mask)


for seq in ids_ood:
  seq_mask = [float(i>0) for i in seq]
  amasks_ood.append(seq_mask)

We use scikit-learn's train_test_split() to use 10% of our training
data as a validation set, and then convert all data into
torch.tensors.

In [20]:
(train_inputs, validation_inputs,
 train_labels, validation_labels) = train_test_split(ids_train, labels_train,
                                                     random_state=42,
                                                     test_size=0.1)
(train_masks, validation_masks,
 _, _) = train_test_split(amasks_train, ids_train,
                          random_state=42, test_size=0.1)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks  = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks  = torch.tensor(validation_masks)
test_inputs = torch.tensor(ids_test)
test_labels = torch.tensor(labels_test)
test_masks  = torch.tensor(amasks_test)
ood_inputs = torch.tensor(ids_ood)
ood_labels = torch.tensor(labels_ood)
ood_masks  = torch.tensor(amasks_ood)



Next we create PyTorch *DataLoader*s for all data sets.
For fine-tuning BERT on a specific task, the authors recommend a
batch size of 16 or 32.

In [21]:
BATCH_SIZE = 16

print('\nDatasets:')
print('Train: ', end="")
train_data = TensorDataset(train_inputs, train_masks,
                           train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler,
                              batch_size=BATCH_SIZE)
print(len(train_data), 'reviews')

print('Validation: ', end="")
validation_data = TensorDataset(validation_inputs, validation_masks,
                                validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,
                                   sampler=validation_sampler,
                                   batch_size=BATCH_SIZE)
print(len(validation_data), 'reviews')

print('Test: ', end="")
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler,
                             batch_size=BATCH_SIZE)
print(len(test_data), 'reviews')


print('OOD: ', end="")
ood_data = TensorDataset(ood_inputs, ood_masks, ood_labels)
ood_sampler = SequentialSampler(ood_data)
ood_dataloader = DataLoader(ood_data, sampler=ood_sampler,
                             batch_size=BATCH_SIZE)
print(len(ood_data), 'reviews')


Datasets:
Train: 49118 reviews
Validation: 5458 reviews
Test: 13645 reviews
OOD: 2729 reviews


BERT MODEL INITIALIZATION

We now load a pretrained BERT model with a single linear
classification layer added on top.


In [22]:
model = BertForSequenceClassification.from_pretrained(BERTMODEL,
                                                      num_labels=2,
                                                      output_hidden_states=True)


model.cuda()
print('\nPretrained BERT model "{}" loaded'.format(BERTMODEL))


100%|██████████| 440473133/440473133 [00:07<00:00, 59745985.97B/s]



Pretrained BERT model "bert-base-uncased" loaded



We set the remaining hyperparameters needed for fine-tuning the
pretrained model: 
 * EPOCHS: the number of training epochs in fine-tuning
   (recommended values between 2 and 4) 
 * WEIGHT_DECAY: weight decay for the Adam optimizer 
 * LR: learning rate for the Adam optimizer 
   (2e-5 to 5e-5 recommended) 
 * WARMUP_STEPS: number of warmup steps to (linearly) reach the
   set learning rate

 We also need to grab the training parameters from the pretrained
 model.

In [23]:
EPOCHS = 4
WEIGHT_DECAY = 0.01
LR = 2e-5
WARMUP_STEPS =int(0.2*len(train_dataloader))

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)],
     'weight_decay': WEIGHT_DECAY},
    {'params': [p for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)],
     'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=LR, eps=1e-8)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS,
                                 t_total=len(train_dataloader)*EPOCHS)

LEARNING

Let's now define functions to train() and evaluate() the model:

In [24]:
def train(epoch, loss_vector=None, log_interval=200):
    # Set model to training mode
    model.train().to(device)

    # Loop over each batch from the training set
    for step, batch in enumerate(train_dataloader):
        # Copy data to GPU if needed
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

        # Zero gradient buffers
        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            # Forward pass
            loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,
                         labels=b_labels)[0]

        if loss_vector is not None:
            loss_vector.append(loss.item())

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()
        scheduler.step()

        # Clear unused variables
        del  b_input_mask, b_labels

        if step % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, step * len(b_input_ids), len(train_dataloader.dataset),
                  100. * step / len(train_dataloader), loss.item()))
            
    # Clear unused variables
    del b_input_ids,batch, loss


In [25]:
def evaluate(loader):
  model.eval()

  n_correct, n_all = 0, 0

  for batch in loader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)
      logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    predictions = np.argmax(logits, axis=1)

    labels = b_labels.to('cpu').numpy()
    n_correct += np.sum(predictions == labels)
    n_all += len(labels)

  print('Accuracy: [{}/{}] {:.4f}'.format(n_correct, n_all,
                                          n_correct/n_all))



    

Now we are ready to train our model using the train()
function. After each epoch, we evaluate the model using the
validation set and evaluate().

In [26]:
train_lossv = []
for epoch in range(1, EPOCHS + 1):
    print()
    train(epoch, train_lossv)
    print('\nValidation set:')
    evaluate(validation_dataloader)

/usr/local/lib/python3.9/dist-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


Train Epoch: 1 [0/49118 (0%)]	Loss: 0.635975
Train Epoch: 1 [3200/49118 (7%)]	Loss: 0.667433
Train Epoch: 1 [6400/49118 (13%)]	Loss: 0.347848
Train Epoch: 1 [9600/49118 (20%)]	Loss: 0.174999
Train Epoch: 1 [12800/49118 (26%)]	Loss: 0.688814
Train Epoch: 1 [16000/49118 (33%)]	Loss: 0.290155
Train Epoch: 1 [19200/49118 (39%)]	Loss: 0.280039
Train Epoch: 1 [22400/49118 (46%)]	Loss: 0.293202
Train Epoch: 1 [25600/49118 (52%)]	Loss: 0.420386
Train Epoch: 1 [28800/49118 (59%)]	Loss: 0.176988
Train Epoch: 1 [32000/49118 (65%)]	Loss: 0.080748
Train Epoch: 1 [35200/49118 (72%)]	Loss: 0.137933
Train Epoch: 1 [38400/49118 (78%)]	Loss: 0.240796
Train Epoch: 1 [41600/49118 (85%)]	Loss: 0.417695
Train Epoch: 1 [44800/49118 (91%)]	Loss: 0.044062
Train Epoch: 1 [48000/49118 (98%)]	Loss: 0.052418

Validation set:
Accuracy: [5004/5458] 0.9168

Train Epoch: 2 [0/49118 (0%)]	Loss: 0.162340
Train Epoch: 2 [3200/49118 (7%)]	Loss: 0.143102
Train Epoch: 2 [6400/49118 (13%)]	Loss: 0.358092
Train Epoch: 2 [9600

Let's take a look at our training loss over all batches:

In [27]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_lossv, label='original')
plt.plot(np.convolve(train_lossv, np.ones(101), 'same') / 101,
         label='averaged')
plt.legend(loc='best')
plt.savefig("training-loss.png")
plt.show()


Inference

For a better measure of the quality of the model, let's see the
model accuracy for the test reviews.

In [28]:
print('\nTest set:')
evaluate(test_dataloader)
# eof


Test set:
Accuracy: [12742/13645] 0.9338


# OOD detection 

Extract BERT logits, features and predictions

In [29]:
def extract_bert_features(loader):
    model.eval()

    last_layer_list = []
    logits_list = []
    softmax_list = []
    label_list = []
    pred_list = []

    for batch in loader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask)
            last_layer = outputs[1][-1][:,0,:]
            logits = outputs[0][:,:]
            predictions = np.argmax(logits.detach().cpu().numpy(), axis=1)
            softmax = torch.nn.functional.softmax(logits, dim=-1)

        last_layer_list.append(last_layer)
        logits_list.append(logits.cpu().numpy())
        softmax_list.append(softmax.cpu().numpy())
        label_list.append(b_labels.cpu().numpy())
        pred_list.append(predictions)
    

        

    last_layer = torch.cat(last_layer_list, dim=0)
    logits = np.concatenate(logits_list, axis=0)
    softmax = np.concatenate(softmax_list, axis=0)
    labels = np.concatenate(label_list, axis=0)
    predictions = np.concatenate(pred_list, axis=0)

    return logits, last_layer.to('cpu').numpy(),  softmax, labels, predictions
    



Optimized memory version to be tested

In [30]:
"""

def extract_bert_features(loader):
    model.eval()

    last_layer_list = []
    logits_list = []
    softmax_list = []
    label_list = []
    pred_list = []

    with torch.no_grad():
        for batch in loader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask)
            last_layer = outputs[-1]
            logits = outputs[0].detach().cpu().numpy()
            predictions = np.argmax(logits, axis=1)
            softmax = torch.nn.functional.softmax(outputs[0], dim=-1).cpu().numpy()

            last_layer_list.append(last_layer.cpu())
            logits_list.append(logits)
            softmax_list.append(softmax)
            label_list.append(b_labels.detach().cpu().numpy())
            pred_list.append(predictions)

    last_layer = torch.cat(last_layer_list, dim=0).numpy()
    labels = np.concatenate(label_list, axis=0)
    predictions = np.concatenate(pred_list, axis=0)

    return last_layer.to('cpu').numpy(), logits_list, softmax_list, labels, predictions

    """


"\n\ndef extract_bert_features(loader):\n    model.eval()\n\n    last_layer_list = []\n    logits_list = []\n    softmax_list = []\n    label_list = []\n    pred_list = []\n\n    with torch.no_grad():\n        for batch in loader:\n            batch = tuple(t.to(device) for t in batch)\n            b_input_ids, b_input_mask, b_labels = batch\n\n            outputs = model(b_input_ids, token_type_ids=None,\n                            attention_mask=b_input_mask)\n            last_layer = outputs[-1]\n            logits = outputs[0].detach().cpu().numpy()\n            predictions = np.argmax(logits, axis=1)\n            softmax = torch.nn.functional.softmax(outputs[0], dim=-1).cpu().numpy()\n\n            last_layer_list.append(last_layer.cpu())\n            logits_list.append(logits)\n            softmax_list.append(softmax)\n            label_list.append(b_labels.detach().cpu().numpy())\n            pred_list.append(predictions)\n\n    last_layer = torch.cat(last_layer_list, dim=0).nu

Now applying on the data sets

In [31]:
train_features = extract_bert_features(train_dataloader)
test_features = extract_bert_features(test_dataloader)
ood_features = extract_bert_features(ood_dataloader)

## OOD with Mahalanobis distance using last layer logits and features

In [32]:
#define the function that calculates the metrics 
def metrics(scores: np.ndarray, labels: np.ndarray, threshold: float):
    """Computes the number of
        * true positives,
        * false positives,
        * true negatives,
        * false negatives,
        * the accuracy
        * false positive rate
        * taux d'erreur
    for a given threshold

    """
    pos = np.where(scores >= threshold) 
    neg = np.where(scores < threshold)
    n_pos = len(pos[0])
    n_neg = len(neg[0])

    tp = np.sum(labels[pos])
    fp = n_pos - tp
    fn = np.sum(labels[neg])
    tn = n_neg - fn

    FPR = fp / (fp + tn)
    Accuracy = (tp+tn)/len(scores)
    ERR = 1 - Accuracy
    return FPR, ERR

In [33]:
def metrics(scores: np.ndarray, labels: np.ndarray, threshold: float):
    """Computes the number of
        * true positives,
        * false positives,
        * true negatives,
        * false negatives,
        * the accuracy
        * false positive rate
        * taux d'erreur
    for a given threshold

    """
    pos = np.where(scores >= threshold) 
    neg = np.where(scores < threshold)
    n_pos = len(pos[0])
    n_neg = len(neg[0])

    tp = np.sum(labels[pos])
    fp = n_pos - tp
    fn = np.sum(labels[neg])
    tn = n_neg - fn

    TPR = tp / (tp + fn)
    FPR = fp / (fp + tn)
    Accuracy = (tp+tn)/len(scores)
    ERR = 1 - Accuracy
    
    # Calculate the new threshold for a TPR of 95%
    new_TPR = 0.95
    new_threshold = np.percentile(scores, (1 - new_TPR) * 100)
    
    # Calculate the new FPR value that is consistent with TPR = 95%
    new_pos = np.where(scores >= new_threshold) 
    new_neg = np.where(scores < new_threshold)
    new_n_pos = len(new_pos[0])
    new_n_neg = len(new_neg[0])
    new_tp = np.sum(labels[new_pos])
    new_fp = new_n_pos - new_tp
    new_fn = np.sum(labels[new_neg])
    new_tn = new_n_neg - new_fn
    new_FPR = new_fp / (new_fp + new_tn)

    return new_FPR, FPR, ERR


Covariance matrix for the latent features of the internal distribution (SST2)

In [34]:
#mean, Covariance matrix and inverse for the latent features of the internal distribution (SST2) 
                  #for logits#                          #for features#
train_mean = [np.mean(train_features[0], axis=0)  ,np.mean(train_features[1], axis=0)]

                  #for logits#                           #for features#
train_cov = [np.cov(train_features[0], rowvar=False), np.cov(train_features[1], rowvar=False)] 

                   #for logits#                          #for features#
train_inv_cov = [np.linalg.inv(train_cov[0]), np.linalg.inv(train_cov[1])] 


In [35]:
for i in range(2):
 
  #Calculation of the Mahalanobis distance on the in-ds and ood data
  inds_test_scores = []
  ood_test_scores = []

  for feature in test_features[i]:
    score = mahalanobis(feature, train_mean[i], train_inv_cov[i])
    inds_test_scores.append(score)

  for feature in ood_features[i]:
    score = mahalanobis(feature, train_mean[i], train_inv_cov[i])
    ood_test_scores.append(score)

  #We will evaluate the performance of OOD detection using the AUROC,
  # AUPR, FPR and ERR metrics
  
  labels = np.concatenate([np.zeros(len(inds_test_scores)), np.ones(len(ood_test_scores))])
  scores = np.concatenate([inds_test_scores, ood_test_scores])

  #define the threshold for ood detection
  threshold = np.mean(inds_test_scores) +  np.std(inds_test_scores)

  #Calculation of metrics
  if i == 0:
    use = 'logits'
  else:
      use = 'features'
  new_FPR, FPR, ERR = metrics (scores, labels, threshold)
  auroc = roc_auc_score(labels, scores)
  aupr = average_precision_score(labels, scores)

  
  print("metrics in ood detection with Mahalanibos distance using "+use+" of last layer")
  print('AUROC:', auroc)
  print('AUPR:', aupr)
  print('FPR_95%:', new_FPR)
  print('FPR:', FPR)
  print('ERR:', ERR)



  #plot of auroc curve and aupr curve
  fpr, tpr, _ = roc_curve(labels, scores)
  #precision, recall, _ = precision_recall_curve(labels, scores)
  

  plt.figure(figsize=(7, 7))
  
  plt.plot(fpr, tpr)
  plt.plot([0, 1], [0, 1], linestyle='--')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('AUROC Curve')



  plt.tight_layout()
  plt.savefig("auroc_mahalabinos_"+use+".png")
  


metrics in ood detection with Mahalanibos distance using logits of last layer
AUROC: 0.39154410219564006
AUPR: 0.14691499469114044
FPR_95%: 0.9739831440087944
FPR: 0.12883840234518137
ERR: 0.25393917185782333
metrics in ood detection with Mahalanibos distance using features of last layer
AUROC: 0.7877837501498837
AUPR: 0.371944922348102
FPR_95%: 0.9402711615976548
FPR: 0.14913887871015025
ERR: 0.20642481983632588


## OOD detection using Energy-based score


In [36]:
# define the energy-based score function
def energy_score(data:np.ndarray):
  energy = np.log(np.sum(np.exp(data), axis=1))
  
  return -energy
#we only use logits
in_scores = energy_score(test_features[0])
out_scores = energy_score(ood_features[0])

labels = np.concatenate([np.zeros(len(in_scores)), np.ones(len(out_scores))])
scores = np.concatenate([in_scores, out_scores])

  #define the threshold for ood detection
threshold = np.mean(in_scores) +  np.std(in_scores)

In [37]:
new_FPR,FPR, ERR = metrics (scores, labels, threshold)
auroc = roc_auc_score(labels, scores)
aupr = average_precision_score(labels, scores)

  
print("metrics in ood detection with Energy-based score using logits of last layer")
print('AUROC:', auroc)
print('AUPR:', aupr)
print('FPR_95%:', new_FPR)
print('FPR:', FPR)
print('ERR:', ERR)

#plot of auroc curve and aupr curve
fpr, tpr, _ = roc_curve(labels, scores)

  

plt.figure(figsize=(7, 7))
  
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUROC Curve')
plt.savefig("auroc_energy.png")

metrics in ood detection with Energy-based score using logits of last layer
AUROC: 0.7977244935542289
AUPR: 0.3590861578119161
FPR_95%: 0.9403444485159399
FPR: 0.16584829607914986
ERR: 0.2112495419567607


## OOD Detection using Maximum Softmax Probability

In [38]:
def MSP_score(data:np.ndarray):
  "inputs are softmax extract from BERT"
  score = np.max(data, axis=1)
  return score

#we calculate the scores for in and out data sets
in_scores = energy_score(test_features[2])
out_scores = energy_score(ood_features[2])

labels = np.concatenate([np.zeros(len(in_scores)), np.ones(len(out_scores))])
scores = np.concatenate([in_scores, out_scores])

  #define the threshold for ood detection
threshold = np.mean(in_scores) +  np.std(in_scores)


In [39]:
new_FPR,FPR, ERR = metrics (scores, labels, threshold)
auroc = roc_auc_score(labels, scores)
aupr = average_precision_score(labels, scores)

  
print("metrics in ood detection with Energy-based score using logits of last layer")
print('AUROC:', auroc)
print('AUPR:', aupr)
print('FPR_95%:', new_FPR)
print('FPR:', FPR)
print('ERR:', ERR)

#plot of auroc curve and aupr curve
fpr, tpr, _ = roc_curve(labels, scores)

  

plt.figure(figsize=(7, 7))
  
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUROC Curve')
plt.savefig("auroc_MSP.png")

metrics in ood detection with Energy-based score using logits of last layer
AUROC: 0.7920021655760683
AUPR: 0.38316313612171865
FPR_95%: 0.9402711615976548
FPR: 0.08501282521069989
ERR: 0.18278978868938556
